# Last iterate convergenceを保証するPrimal dual法

参考：
* [ReLOAD: Reinforcement Learning with Optimistic Ascent-Descent for Last-Iterate Convergence in Constrained MDPs](https://arxiv.org/abs/2302.01275)
* [Last-Iterate Convergent Policy Gradient Primal-Dual Methods for Constrained MDPs](https://arxiv.org/abs/2306.11700)：こっちをベースにします

CMDPにおいて，単純にprimal-dual法を実現しても実行可能な方策は得られません．
簡単な手法は過去の方策の期待値を取る方法ですが，これはNNなどの関数近似が入ると難しいです．

今回は方策勾配法を使ってlast-iterate convergenceを実現する方法を見てみましょう．

表記：

* 制約付きMDP：$\underset{\pi \in \Pi}{\operatorname{maximize}} V_r^\pi(\rho) \quad \text { subject to } V_u^\pi(\rho) \geq b$：
* $g:=u-(1-\gamma) b$とする
* $L(\pi, \lambda):=V_r^\pi(\rho)+\lambda V_g^\pi(\rho)$
* ラグランジュ形式：$\underset{\pi \in \Pi}{\operatorname{maximize}} \underset{\lambda \in[0, \infty]}{\operatorname{minimize}} V_{r+\lambda g}^\pi(\rho)$
    * 鞍点$(\pi', \lambda')$：$V_{r+\lambda^{\prime} g}^\pi(\rho) \leq V_{r+\lambda^{\prime} g}^{\pi^{\prime}}(\rho) \leq V_{r+\lambda g}^{\pi^{\prime}}(\rho)$


**Last iterate convergenceの難しさ：**

大きく３つあります
1. Max側とMin側が非対称です．方策側は遷移についての最大化を目的としますが，$\lambda$側は報酬を最小化します．
2. 問題自体が非凸です． 
3. 全ての状態行動について最適な方策が鞍点にならないことがあります．

## Last-iterate convergenceを達成するアルゴリズム


次のように正則化されたラグランジアンを考えます：

$$L_\tau(\pi, \lambda):=V_{r+\lambda g}^\pi(\rho)+\tau\left(\mathcal{H}(\pi)+\frac{1}{2} \lambda^2\right)$$

ここで，$\mathcal{H}(\pi):=(1-\gamma) \mathbb{E}\left[\sum_{t=0}^{\infty}-\gamma^t \log \pi\left(a_t \mid s_t\right)\right]$です．

このラグランジアンには，次を満たす鞍点$(\bar{\pi}, \bar{\lambda}) \in \Pi \times \Lambda$が存在し，それは唯一存在します．
任意の$\pi \in \Pi$と$\lambda \in \Lambda$について，

$$
L_\tau(\bar{\pi}, \lambda) \geq L_\tau(\bar{\pi}, \bar{\lambda}) \geq L_\tau(\pi, \bar{\lambda})
$$

**証明**：Occupancy measureに直して適当に書けばいけます．


---

この唯一の鞍点を

* $\pi_\tau^{\star}=\operatorname{argmax}_{\pi \in \Pi} \min _{\lambda \in \Lambda} L_\tau(\pi, \lambda)$
* $\lambda_\tau^{\star}=\operatorname{argmin}_{\lambda \in \Lambda} \max _{\pi \in \Pi} L_\tau(\pi, \lambda)$

としましょう．このとき，次が成立します．

$$
V_{r+\lambda_\tau^{\star} g}^\pi(\rho)-\tau \mathcal{H}\left(\pi_\tau^{\star}\right) \leq V_{r+\lambda_\tau^{\star} g}^{\pi_\tau^{\star}}(\rho) \leq V_{r+\lambda g}^{\pi_\tau^{\star}}(\rho)+\frac{\tau}{2} \lambda^2 \text { for all }(\pi, \lambda) \in \Pi \times \Lambda
$$

つまり，正則化された鞍点である$\left(\pi_\tau^{\star}, \lambda_\tau^{\star}\right)$は，もとのラグランジアンである$V_{r+\lambda g}^\pi(\rho)$について，
* $\frac{\tau}{2} \lambda^2$
* $\tau \mathcal{H}\left(\pi_\tau^{\star}\right)$

の２つについて誤差が生じた鞍点です．
ここで，次のような更新を考えます

$$
\begin{aligned}
\pi_{t+1}(\cdot \mid s) & =\underset{\pi(\cdot \mid s) \in \Delta(A)}{\operatorname{argmax}}\left\{\sum_a \pi(a \mid s) Q_{r+\lambda_t g+\tau \psi_t}^{\pi_t}(s, a)-\frac{1}{\eta} \mathrm{KL}\left(\pi(\cdot \mid s), \pi_t(\cdot \mid s)\right)\right\} \\
\lambda_{t+1} & =\underset{\lambda \in \Lambda}{\operatorname{argmin}}\left\{\lambda\left(V_g^{\pi_t}(\rho)+\tau \lambda_t\right)+\frac{1}{2 \eta}\left(\lambda-\lambda_t\right)^2\right\},
\end{aligned}
$$

つまり，
* 方策側は$Q_{r+\lambda_t g+\tau \psi_t}^{\pi_t}(s, a)$の方向に対してMirror descentで更新します．
* $\lambda$側はprojected gradient descentで更新しています．

## 解析

次が成立します．

---

**定理**

* $\mathrm{KL}_t(\rho):=\sum_s d_\rho^{\pi_\tau^{\star}}(s) \mathrm{KL}_t(s)$ 
* $\mathrm{KL}_t(s):=\mathrm{KL}\left(\pi_\tau^{\star}(\cdot \mid s), \pi_t(\cdot \mid s)\right)$
* $\Phi_t=\mathrm{KL}_t(\rho)+\frac{1}{2}\left(\lambda_\tau^{\star}-\lambda_t\right)^2$

として，$\Phi_t$を$\pi_t$と$\lambda_t$の非最適性とします．
このとき，次が成立します：

$$
\Phi_{t+1} \leq \mathrm{e}^{-\eta \tau t} \Phi_1+\frac{\eta}{\tau} \max \left(\left(C_{\tau, \xi}\right)^2,\left(C_{\tau, \xi}^{\prime}\right)^2\right)
$$

ここで，
$C_{\tau, \xi}:=(1+1 /((1-\gamma) \xi)+\tau \log |A|) /(1-\gamma)$ 
および$C_{\tau, \xi}^{\prime}:=(1+\tau / \xi) /(1-\gamma)$
です．

---

* 一項目は$t\to \infty$で線形に小さくなります．
* 二項目は学習率$\eta$と$\tau$を調整することで小さくできます．

実際，$\eta=\epsilon \tau$とすれば，$t \geq\left(1 /\left(\epsilon \tau^2\right)\right) \log (1 / \epsilon)$回の更新で$\Phi_t=O(\epsilon)$に収束します．


### 証明

まず，双対ギャップを次のように分解します：

$$
L_\tau\left(\pi_\tau^{\star}, \lambda_t\right)-L_\tau\left(\pi_t, \lambda_\tau^{\star}\right)=\underbrace{L_\tau\left(\pi_\tau^{\star}, \lambda_t\right)-L_\tau\left(\pi_t, \lambda_t\right)}_{(\mathrm{i})}+\underbrace{L_\tau\left(\pi_t, \lambda_t\right)-L_\tau\left(\pi_t, \lambda_\tau^{\star}\right)}_{(\mathrm{ii})}
$$

**(i)項目のバウンド**

ここで，エントロピーは次で書けることに注意します．

$$
\mathcal{H}(\pi):=(1-\gamma) \mathbb{E}\left[\sum_{t=0}^{\infty}-\gamma^t \log \pi\left(a_t \mid s_t\right)\right]=-\sum_{s, a} d_\rho^\pi(s) \pi(a \mid s) \log \pi(a \mid s)
$$

すると，最初の項は

$$
\begin{aligned}
&L_\tau\left(\pi_\tau^{\star}, \lambda_t\right)-L_\tau\left(\pi_t, \lambda_t\right) \\
= & V_{r+\lambda_t g}^{\pi_\tau^{\star}}(\rho)-V_{r+\lambda_t g}^{\pi_t}(\rho) \\
& -\tau \sum_{s, a} d_\rho^{\pi_\tau^{\star}}(s) \pi_\tau^{\star}(a \mid s) \log \pi_\tau^{\star}(a \mid s)+\tau \sum_{s, a} d_\rho^{\pi_t}(s) \pi_t(a \mid s) \log \pi_t(a \mid s) \\
= & V_{r+\lambda_t g+\tau \psi_t}^{\pi_\tau^{\star}}(\rho)-V_{r+\lambda_t g+\tau \psi_t}^{\pi_t}(\rho)-\tau V_{\psi_t}^{\pi_\tau^{\star}}(\rho)+\tau V_{\psi_t}^{\pi_t}(\rho) \\
& -\tau \sum_{s, a} d_\rho^{\pi_\tau^{\star}}(s) \pi_\tau^{\star}(a \mid s) \log \pi_\tau^{\star}(a \mid s)+\tau \sum_{s, a} d_\rho^{\pi_t}(s) \pi_t(a \mid s) \log \pi_t(a \mid s)\\
\end{aligned}
$$

さらに[RL_useful_lemma.ipynb](RL_useful_lemma.ipynb)の
**補題: Extended Value Difference**を使うと，
$$
\begin{aligned}
= & \sum_{s, a} d_\rho^{\pi_\tau^{\star}}(s)\left(\pi_\tau^{\star}(a \mid s)-\pi_t(a \mid s)\right) Q_{r+\lambda_t g+\tau \psi_t}^{\pi_t}(s, a) \\
& +\tau \sum_{s, a} d_\rho^{\pi_\tau^{\star}}(s) \pi_\tau^{\star}(a \mid s) \log \pi_t(a \mid s)-\tau \sum_{s, a} d_\rho^{\pi_t}(s) \pi_t(a \mid s) \log \pi_t(a \mid s) \\
& -\tau \sum_{s, a} d_\rho^{\pi_\tau^{\star}}(s) \pi_\tau^{\star}(a \mid s) \log \pi_\tau^{\star}(a \mid s)+\tau \sum_{s, a} d_\rho^{\pi_t}(s) \pi_t(a \mid s) \log \pi_t(a \mid s)\\
\end{aligned}
$$

として変形できます．
ここで，[RL_useful_lemma.ipynb](RL_useful_lemma.ipynb)の**補題：Mirror descentをKLで抑える**を使えば，

$$
\begin{aligned}
& =\sum_{s, a} d_\rho^{\pi_\tau^{\star}}(s)\left(\pi_\tau^{\star}(a \mid s)-\pi_t(a \mid s)\right) Q_{r+\lambda_t g+\tau \psi_t}^{\pi_t}(s, a)-\tau \sum_s d_\rho^{\pi_\tau^{\star}}(s) \mathrm{KL}_t(s) \\
& \leq \sum_s d_\rho^{\pi_\tau^{\star}}(s)\left(\frac{\mathrm{KL}_t(s)-\mathrm{KL}_{t+1}(s)}{\eta}\right)+\eta\left(C_{\tau, \xi}\right)^2-\tau \sum_s d_\rho^{\pi_\tau^{\star}}(s) \mathrm{KL}_t(s) \\
& =\sum_s d_\rho^{\pi_\tau^{\star}}(s)\left(\frac{(1-\eta \tau) \mathrm{KL}_t(s)-\mathrm{KL}_{t+1}(s)}{\eta}\right)+\eta\left(C_{\tau, \xi}\right)^2 \\
& =\frac{(1-\eta \tau) \mathrm{KL}_t(\rho)-\mathrm{KL}_{t+1}(\rho)}{\eta}+\eta\left(C_{\tau, \xi}\right)^2 .
\end{aligned}
$$

ここで，$Q_{r+\lambda_t g+\tau \psi_t}^{\pi_t}(s, a) \leq \frac{1}{1-\gamma}\left(1+\frac{1}{(1-\gamma) \xi}+\tau \log |A|\right):=C_{\tau, \xi}$としました．


**(ii)項目のバウンド**

$$
\begin{aligned}
& L_\tau\left(\pi_t, \lambda_t\right)-L_\tau\left(\pi_t, \lambda_\tau^{\star}\right) \\
& =V_{r+\lambda_t g}^{\pi_t}(\rho)-V_{r+\lambda_\tau^{\star} g}^{\pi_t}(\rho)+\frac{1}{2} \tau\left(\lambda_t\right)^2-\frac{1}{2} \tau\left(\lambda_\tau^{\star}\right)^2 \\
& =\left(\lambda_t-\lambda_\tau^{\star}\right) V_g^{\pi_t}(\rho)+\frac{1}{2} \tau\left(\lambda_t\right)^2-\frac{1}{2} \tau\left(\lambda_\tau^{\star}\right)^2\\
& =\left(\lambda_t-\lambda_\tau^{\star}\right)\left(V_g^{\pi_t}(\rho)+\tau \lambda_t\right)-\frac{1}{2} \tau\left(\lambda_t-\lambda_\tau^{\star}\right)^2 \\
& \leq \frac{\left(\lambda_\tau^{\star}-\lambda_t\right)^2-\left(\lambda_\tau^{\star}-\lambda_{t+1}\right)^2}{2 \eta}+\frac{1}{2} \eta\left(C_{\tau, \xi}^{\prime}\right)^2-\frac{1}{2} \tau\left(\lambda_t-\lambda_\tau^{\star}\right)^2 \\
& =\frac{(1-\eta \tau)\left(\lambda_\tau^{\star}-\lambda_t\right)^2-\left(\lambda_\tau^{\star}-\lambda_{t+1}\right)^2}{2 \eta}+\frac{1}{2} \eta\left(C_{\tau, \xi}^{\prime}\right)^2
\end{aligned}
$$

ここで，不等式は$V_g^{\pi_t}(\rho)+\tau \lambda_t \leq \frac{1}{1-\gamma}\left(1+\frac{\tau}{\xi}\right):=C_{\tau, \xi}^{\prime}$と，[勾配降下の話](OPT_gradient.ipynb)を使いました．（ここで$\lambda$の凸性を使ってる？）


定義
$\Phi_t:=\mathrm{KL}_t(\rho)+\frac{1}{2}\left(\lambda_\tau^{\star}-\lambda_t\right)^2$
に従って，上の不等式を合体させれば，

$$
\begin{aligned}
\Phi_{t+1} & \leq(1-\eta \tau) \Phi_t-\eta\left(L_\tau\left(\pi_\tau^{\star}, \lambda_t\right)-L_\tau\left(\pi_t, \lambda_\tau^{\star}\right)\right)+\eta^2 \max \left(\left(C_{\tau, \xi}\right)^2,\left(C_{\tau, \xi}^{\prime}\right)^2\right) \\
& \leq(1-\eta \tau) \Phi_t+\eta^2 \max \left(\left(C_{\tau, \xi}\right)^2,\left(C_{\tau, \xi}^{\prime}\right)^2\right) .
\end{aligned}
$$

が得られます．これを再帰的に繰り返して，

$$
\begin{aligned}
\Phi_{t+1} & \leq(1-\eta \tau) \Phi_t+\eta^2 \max \left(\left(C_{\tau, \xi}\right)^2,\left(C_{\tau, \xi}^{\prime}\right)^2\right) \\
& \leq(1-\eta \tau)^2 \Phi_{t-1}+\left(\eta^2+\eta^2(1-\eta \tau)\right) \max \left(\left(C_{\tau, \xi}\right)^2,\left(C_{\tau, \xi}^{\prime}\right)^2\right) \\
& \leq \cdots \\
& \leq(1-\eta \tau)^t \Phi_1+\left(\eta^2\left(1+(1-\eta \tau)+(1-\eta \tau)^2+\cdots\right)\right) \max \left(\left(C_{\tau, \xi}\right)^2,\left(C_{\tau, \xi}^{\prime}\right)^2\right) \\
& \leq(1-\eta \tau)^t \Phi_1+\frac{\eta}{\tau} \max \left(\left(C_{\tau, \xi}\right)^2,\left(C_{\tau, \xi}^{\prime}\right)^2\right) \\
& \leq \mathrm{e}^{-\eta \tau t} \Phi_1+\frac{\eta}{\tau} \max \left(\left(C_{\tau, \xi}\right)^2,\left(C_{\tau, \xi}^{\prime}\right)^2\right)
\end{aligned}
$$

を得ます．


## $\Phi$からsub-optimalityへの変換

次が成立します．

---

十分小さい$\epsilon > 0$について，$\eta=\Theta\left(\epsilon^4\right)$
および
$\tau=\Theta\left(\epsilon^2\right)$
のとき，

$$
V_r^{\pi^{\star}}(\rho)-V_r^{\pi_t}(\rho) \leq \epsilon \text { and }-V_g^{\pi_t}(\rho) \leq \epsilon \text { for any } t=\Omega\left(\frac{1}{\epsilon^6} \log ^2 \frac{1}{\epsilon}\right)
$$

が成立します．

**証明**

$\tau=\Theta(\epsilon)$ と$\eta=\Theta\left(\epsilon^2\right)$のとき，$t=\Omega\left(\frac{1}{\epsilon^3} \log \frac{1}{\epsilon}\right)$について，$\Phi_{t+1}=O(\epsilon)$であることはすぐにわかります．

このような$t$について，$\left(\pi_t, \lambda_t\right)$を考えましょう．

$\Phi$の定義から，
$$\mathrm{KL}_t(\rho)=O(\epsilon)\; \text{ and } \frac{1}{2}\left(\lambda_\tau^{\star}-\lambda_t\right)^2=O(\epsilon)$$
です．

suboptimalityをバウンドしましょう．まず，

$$
V_r^{\pi^{\star}}(\rho)-V_r^{\pi_t}(\rho)=\underbrace{V_r^{\pi^{\star}}(\rho)-V_r^{\pi_\tau^{\star}}(\rho)}_{(\mathrm{i})}+\underbrace{V_r^{\pi_\tau^{\star}}(\rho)-V_r^{\pi_t}(\rho)}_{(\mathrm{ii})} .
$$

です．

**(ii)のバウンド**

$$
\begin{aligned}
\text { (ii) } & =\sum_{s, a} d_\rho^{\pi_\tau^{\star}}(s)\left(\pi_\tau^{\star}(a \mid s)-\pi_t(a \mid s)\right) Q_r^{\pi_t}(s, a) \\
& \leq \frac{1}{1-\gamma} \sum_s d_\rho^{\pi_\tau^{\star}}(s)\left\|\pi_\tau^{\star}(\cdot \mid s)-\pi_t(\cdot \mid s)\right\|_1 \\
& \leq \frac{1}{1-\gamma} \sum_s d_\rho^{\pi_\tau^{\star}}(s) \sqrt{2 \mathrm{KL}_t(s)} \\
& \leq \frac{1}{1-\gamma} \sqrt{2 \sum_s d_\rho^{\pi_\tau^{\star}}(s) \mathrm{KL}_t(s)} \\
& =\frac{1}{1-\gamma} \sqrt{2 \mathrm{KL}_t(\rho)}
\end{aligned}
$$

が成り立ちます．
$\mathrm{KL}_t(\rho)=O(\epsilon)$なので，(ii)$\leq O(\sqrt{\epsilon})$です．

**(i)のバウンド**

エントロピー正則化の性質から，
$$
V_r^{\pi^{\star}}(\rho)-\tau \mathcal{H}\left(\pi_\tau^{\star}\right) \leq V_r^{\pi_\tau^{\star}}(\rho)+\lambda_\tau^{\star}\left(V_g^{\pi_\tau^{\star}}(\rho)-V_g^{\pi^{\star}}(\rho)\right)
$$

であり，これを利用すると

$$
(\mathrm{i})=V_r^{\pi^{\star}}(\rho)-V_r^{\pi_\tau^{\star}}(\rho) \leq \tau \mathcal{H}\left(\pi_\tau^{\star}\right)
$$

がすぐにわかります．$\tau=\Theta(\epsilon)$とすれば$V_r^{\pi^{\star}}(\rho)-V_r^{\pi_t}(\rho) \leq O(\sqrt{\epsilon})$が得られます．